# $\text{LiFePO}_4$ System

Adapted from: https://sunnysuite.github.io/Sunny.jl/stable/examples/spinw/SW13_LiNiPO4.html
The example is changed to $\text{LiFePO}_4$.

### Crystal Cell

In [1]:
using Sunny, GLMakie

units = Units(:meV, :angstrom);

The unit cell has the following lattice vectors (from Toft-Petersen et. al.)


|     |  a [Å]  |  b [Å]  |  c [Å]  |
| --- | --- | --- | --- |
| LiFePO4 | 10.337 | 6.011 | 4.695 | 

In [2]:
latvecs = lattice_vectors(10.337, 6.011, 4.695, 90, 90, 90)

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 10.337  0.0    0.0
  0.0    6.011  0.0
  0.0    0.0    4.695

In [3]:
cryst = Crystal(latvecs, [[1/4, 1/4, 0]], 62, types = ["Fe"])
# QUESTION: Spacegroup 62?

Crystal
Spacegroup 'P n m a' (62)
Lattice params a=10.34, b=6.011, c=4.695, α=90°, β=90°, γ=90°
Cell volume 291.7
Type 'Fe', Wyckoff 4c (site sym. '.m.'):
   1. [1/4, 1/4, 0]
   2. [3/4, 3/4, 0]
   3. [3/4, 1/4, 1/2]
   4. [1/4, 3/4, 1/2]


In [4]:
view_crystal(cryst)

### Spin System

In [5]:
# QUESTION: Correct values for s and g?
sys = System(cryst, [1 => Moment(s=2, g=2)], :dipole)
# QUESTION: Why dipole_uncorrected?
# ANSWER: "The mode :dipole_uncorrected avoids a classical-to-quantum rescaling factor 
# of anisotropy strengths, as needed for consistency with the original fits [of the data].

System [Dipole mode]
Supercell (1×1×1)×4
Energy per site 0


The interaction interaction energies need to be set.
(values from Yiu et. al. and Toft-Petersen et. al.)

In [6]:
this_study = false  # QUESTION: which should i use?
if this_study 
	Jbc = 0.46
	Jb  = 0.09
	Jc  = 0.1
	Jab = 0.09
	Jac = 0.01
	Da  = 0.86
	Db  = 0.00  # easy axis
	Dc  = 2.23
	Hmf = 3.69
else
	Jbc = 0.77
	Jb  = 0.30
	Jc  = 0.14
	Jab = 0.14
	Jac = 0.05
	Da  = 0.62
	Db  = 0.00  # easy axis
	Dc  = 1.56
	Hmf = 4.84
end
set_exchange!(sys, Jbc, Bond(2, 3, [ 0, 0, 0]))
set_exchange!(sys, Jc , Bond(1, 1, [ 0, 0, 1]))
set_exchange!(sys, Jac, Bond(2, 4, [ 0, 0, 0]))
set_exchange!(sys, Jac, Bond(1, 3, [ 0, 0, 0]))
set_exchange!(sys, Jab, Bond(3, 4, [ 0, 0, 0]))
set_exchange!(sys, Jab, Bond(1, 2, [ 0, 0, 0]))
set_exchange!(sys, Jb , Bond(1, 1, [ 0, 1, 0]))


set_onsite_coupling!(sys, S -> Da*S[1]^2 + Db*S[2]^2 + Dc*S[3]^2, 1)

# QUESTION: What about the missing 3 interactions?

In [7]:
view_crystal(sys)

### Optimizing spins
We can now minimize the energy

In [8]:
for _ in 1:10
	randomize_spins!(sys)
	minimize_energy!(sys)
	if energy_per_site(sys) < -2.0
		break
	end
end
energy_per_site(sys)

-2.940000000000002

... and plot the spins in the ground state found above

In [9]:
plot_spins(sys; color=[S[2] for S in sys.dipoles])

### Spin wave theory

In [10]:
swt = SpinWaveTheory(sys; measure=ssf_perp(sys))

SpinWaveTheory [Dipole mode]
  4 atoms


We define a path connecting high symmetry points in the reciprocal lattice. 600 points are sampled along this path

In [ ]:
qpaths = [
	q_space_path(cryst, [[ -1, 1, 0], [ 1, 1, 0]], 600),
	q_space_path(cryst, [[ 0, 0, 0], [ 0, 4, 0]], 600),
	q_space_path(cryst, [[ 0, 0, 0], [ 0, 0, 3]], 600),
]

qpaths = [
	q_space_path(cryst, [[ 0,.5, 0], [ 0, 2, 0]], 600),
	q_space_path(cryst, [[ 0, 1, -0.75], [ 0, 1, 2]], 600),
]

2-element Vector{Sunny.QPath}:
 Sunny.QPath (600 samples)
 Sunny.QPath (600 samples)

The intensities can now be found and shown (compare to Yiu et al.)

In [ ]:
ress = [intensities_bands(swt, qpath) for qpath in qpaths]

fig = Figure(size=(1600, 500))

for (i, res) in enumerate(ress)
	ax = plot_intensities!(fig[1, i], res; units, title="LiFePO₄ LSWT")
end
fig